In [1]:
import numpy as np
from PIL import Image

class Sphere:
    def __init__(self, center, radius, color):
        self.center = np.array(center)
        self.radius = radius
        self.color = np.array(color)

    def ray_intersection(self, ray_origin, ray_direction):
        oc = ray_origin - self.center
        a = np.dot(ray_direction, ray_direction)
        b = 2 * np.dot(oc, ray_direction)
        c = np.dot(oc, oc) - self.radius ** 2

        discriminant = b ** 2 - 4 * a * c

        if discriminant < 0:
            return np.inf

        t1 = (-b + np.sqrt(discriminant)) / (2 * a)
        t2 = (-b - np.sqrt(discriminant)) / (2 * a)

        if t1 >= 0 and t2 >= 0:
            return min(t1, t2)
        elif t1 >= 0:
            return t1
        elif t2 >= 0:
            return t2
        else:
            return np.inf

def trace_ray(ray_origin, ray_direction, objects):
    closest_t = np.inf
    closest_object = None

    for obj in objects:
        t = obj.ray_intersection(ray_origin, ray_direction)
        if t < closest_t:
            closest_t = t
            closest_object = obj

    if closest_object is None:
        return np.array([0, 0, 0])
    else:
        hit_point = ray_origin + ray_direction * closest_t
        normal = (hit_point - closest_object.center) / closest_object.radius
        return closest_object.color

# Scene configuration
width = 800
height = 600
fov = np.pi / 3

# Camera configuration
camera_pos = np.array([0, 0, -5])
camera_forward = np.array([0, 0, 1])
camera_right = np.array([1, 0, 0])
camera_up = np.array([0, 1, 0])

# Create spheres
sphere1 = Sphere(center=[0, 0, 0], radius=1, color=[255, 0, 0])
sphere2 = Sphere(center=[-1.5, 0, 2], radius=0.5, color=[0, 255, 0])
sphere3 = Sphere(center=[1.5, 0, 2], radius=0.5, color=[0, 0, 255])
objects = [sphere1, sphere2, sphere3]

image = np.zeros((height, width, 3), dtype=np.uint8)

for y in range(height):
    for x in range(width):
        ray_direction = camera_forward + camera_right * (2 * (x + 0.5) / width - 1) * np.tan(fov / 2) * width / height - camera_up * (2 * (y + 0.5) / height - 1) * np.tan(fov / 2)
        ray_direction = ray_direction / np.linalg.norm(ray_direction)

        color = trace_ray(camera_pos, ray_direction, objects)

        image[y, x] = color

image = Image.fromarray(image)
image.save("output.png")
image.show()
